Here we wish to solve a 1d Burger's equations with Dirchlet boundary conditions. To do this, we begin by importing the useful packages.

In [2]:
using ProgressMeter
using Random
using Statistics
using UniqueKronecker
using LinearAlgebra
using RLinearAlgebra
using PolynomialModelReductionDataset
using LiftAndLearn
const poly_dat = PolynomialModelReductionDataset
const LnL = LiftAndLearn

LiftAndLearn

Now we use the PolynomialModelReductionDataset to define our Burger's model. This model will be conisdered over $2^13$ spatial points with a time stepping spacing of $1\times 10^{-3}$.

In [15]:
num_inputs = 10;
r = 10;
Ω = (0.0, 10.0);
Nx = 2^11; dt = 0.25e-3;
burger = poly_dat.BurgersModel(
    spatial_domain=Ω, time_domain=(0.0, 1.0), Δx=(Ω[2] + 1/Nx)/Nx, Δt=dt,
    diffusion_coeffs=range(0.1, 1.0, length=10), BC=:dirichlet,
);

We also define options for the operator inference portion of the solution.

In [16]:
options = LnL.LSOpInfOption(
    system=LnL.SystemStructure(
        state=[1,2],
        control=1,
        output=1
    ),
    vars=LnL.VariableStructure(
        N=1,
    ),
    data=LnL.DataStructure(
        Δt=dt,
        deriv_type="SI"
    ),
    optim=LnL.OptimizationSetting(
        verbose=true,
    ),
);
Utest = ones(burger.time_dim, 1);  # Reference input/boundary condition for OpInf testing 

With this defined, we can now generate our testing data for Burger's equations. 

In [17]:
μ = burger.diffusion_coeffs[1]

 ## Create testing data
A, F, B = burger.finite_diff_model(burger, μ)
C = ones(1, burger.spatial_dim) / burger.spatial_dim
Xtest = burger.integrate_model(burger.tspan, burger.IC, Utest; linear_matrix=A,
                                control_matrix=B, quadratic_matrix=F, system_input=true)
Ytest = C * Xtest

op_burger = LnL.Operators(A=A, B=B, C=C, A2u=F)

## training data for inferred dynamical models
Urand = rand(burger.time_dim, num_inputs)
Xall = Vector{Matrix{Float64}}(undef, num_inputs)
Xdotall = Vector{Matrix{Float64}}(undef, num_inputs)
for j in 1:num_inputs
    states = burger.integrate_model(burger.tspan, burger.IC, Urand[:, j], linear_matrix=A,
                                        control_matrix=B, quadratic_matrix=F, system_input=true) 
    Xall[j] = states[:, 2:end]
    Xdotall[j] = (states[:, 2:end] - states[:, 1:end-1]) / burger.Δt
end
X = reduce(hcat, Xall)
R = reduce(hcat, Xdotall)
U = reshape(Urand[2:end,:], (burger.time_dim - 1) * num_inputs, 1)
Y = C * X

1×40000 Matrix{Float64}:
 0.0  -2.71051e-20  -2.71051e-20  0.0  0.0  …  -6.62664e-16  -6.62664e-16

Now that we have generated our testing data, lets look at the scale of the dataset. An consider the runtime of operator inference when we use the truncated SVD.

In [18]:
size(X)

(2048, 40000)

In [19]:
svd_time = @elapsed tmp = svd(X);
svd_time += @elapsed SVD_Vr =  tmp.U[:, 1:r];
soi_time = @elapsed SVD_op_inf = LnL.opinf(X, SVD_Vr, op_burger, options; U=U, Y=Y);
soi_time += @elapsed Finf_extract = UniqueKronecker.extractF(SVD_op_inf.A2u, r);
soi_time += @elapsed sXinf = burger.integrate_model(burger.tspan, SVD_Vr' * burger.IC, Utest; linear_matrix=SVD_op_inf.A[1:r, 1:r],
    control_matrix=SVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
soi_time += @elapsed sYinf = SVD_op_inf.C[1:1, 1:r] * sXinf;

# Compute errors
SVD_PE, SVD_ISE, SVD_IOE, SVD_OSE, SVD_OOE = LnL.compute_all_errors(Xtest, Ytest, sXinf, sYinf, sXinf, sYinf, SVD_Vr);
printstyled("The svd time is: ", svd_time, "\n")
printstyled("The operator inference time is: ", soi_time, "\n")
printstyled("The svd data error is: ", norm(X - SVD_Vr * SVD_Vr'* X),"\n")
printstyled("The svd state error is: ", SVD_OSE, "\n")

The svd time is: 20.088764959
The operator inference time is: 137.83761254299998
The svd data error is: 0.841169958625475
The svd state error is: 0.004212667687397299


Run the same test with the randomizedSVD

With `Gaussian`

In [20]:
rsvd_time = @elapsed rtmp = rapproximate(RandSVD(compressor = Gaussian(compression_dim = r + 2, cardinality = Right())), X);
rsvd_time += @elapsed rSVD_Vr =  rtmp.U[:, 1:r];
roi_time = @elapsed rSVD_op_inf = LnL.opinf(X, rSVD_Vr, op_burger, options; U=U, Y=Y);
roi_time += @elapsed Finf_extract = UniqueKronecker.extractF(rSVD_op_inf.A2u, r);
roi_time += @elapsed rXinf = burger.integrate_model(burger.tspan, rSVD_Vr' * burger.IC, Utest; linear_matrix=rSVD_op_inf.A[1:r, 1:r],
    control_matrix=rSVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
roi_time += @elapsed rYinf = rSVD_op_inf.C[1:1, 1:r] * rXinf;

# Compute errors
rSVD_PE, rSVD_ISE, rSVD_IOE, rSVD_OSE, rSVD_OOE = LnL.compute_all_errors(Xtest, Ytest, rXinf, rYinf, rXinf, rYinf, rSVD_Vr);
printstyled("The total randomized svd time is: ", rsvd_time, "\n")
printstyled("The operator inference time is: ", roi_time, "\n")
printstyled("The randomized svd data error is: ", norm(X - rSVD_Vr * rSVD_Vr'* X),"\n")
printstyled("The randomized svd state error is: ", rSVD_OSE, "\n")

The total randomized svd time is: 0.13467470799999998
The operator inference time is: 135.566517041
The randomized svd data error is: 1.2316805087413263
The randomized svd state error is: 0.004928568981284896


With `SparseSign`

In [21]:
rsvd_time = @elapsed rtmp = rapproximate(RandSVD(compressor = SparseSign(compression_dim = r + 2, cardinality = Right(), nnz = 4)), X);
rsvd_time += @elapsed rSVD_Vr =  rtmp.U[:, 1:r];
roi_time = @elapsed rSVD_op_inf = LnL.opinf(X, rSVD_Vr, op_burger, options; U=U, Y=Y);
roi_time += @elapsed Finf_extract = UniqueKronecker.extractF(rSVD_op_inf.A2u, r);
roi_time += @elapsed rXinf = burger.integrate_model(burger.tspan, rSVD_Vr' * burger.IC, Utest; linear_matrix=rSVD_op_inf.A[1:r, 1:r],
    control_matrix=rSVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
roi_time += @elapsed rYinf = rSVD_op_inf.C[1:1, 1:r] * rXinf;

# Compute errors
rSVD_PE, rSVD_ISE, rSVD_IOE, rSVD_OSE, rSVD_OOE = LnL.compute_all_errors(Xtest, Ytest, rXinf, rYinf, rXinf, rYinf, rSVD_Vr);
printstyled("The total randomized svd time is: ", rsvd_time, "\n")
printstyled("The operator inference time is: ", roi_time, "\n")
printstyled("The randomized svd data error is: ", norm(X - rSVD_Vr * rSVD_Vr'* X),"\n")
printstyled("The randomized svd state error is: ", rSVD_OSE, "\n")

The total randomized svd time is: 0.154586043
The operator inference time is: 136.178715001
The randomized svd data error is: 0.9525984519352487
The randomized svd state error is: 0.0046180197006622575


with `CountSketch`

In [23]:
rsvd_time = @elapsed rtmp = rapproximate(RandSVD(compressor = CountSketch(compression_dim = r + 5,  cardinality = Right())), X);
rsvd_time += @elapsed rSVD_Vr =  rtmp.U[:, 1:r];
roi_time = @elapsed rSVD_op_inf = LnL.opinf(X, rSVD_Vr, op_burger, options; U=U, Y=Y);
roi_time += @elapsed Finf_extract = UniqueKronecker.extractF(rSVD_op_inf.A2u, r);
roi_time += @elapsed rXinf = burger.integrate_model(burger.tspan, rSVD_Vr' * burger.IC, Utest; linear_matrix=rSVD_op_inf.A[1:r, 1:r],
    control_matrix=rSVD_op_inf.B[1:r, :], quadratic_matrix=Finf_extract, system_input=true); # <- use F
roi_time += @elapsed rYinf = rSVD_op_inf.C[1:1, 1:r] * rXinf;

# Compute errors
rSVD_PE, rSVD_ISE, rSVD_IOE, rSVD_OSE, rSVD_OOE = LnL.compute_all_errors(Xtest, Ytest, rXinf, rYinf, rXinf, rYinf, rSVD_Vr);
printstyled("The total randomized svd time is: ", rsvd_time, "\n")
printstyled("The operator inference time is: ", roi_time, "\n")
printstyled("The randomized svd data error is: ", norm(X - rSVD_Vr * rSVD_Vr'* X),"\n")
printstyled("The randomized svd state error is: ", rSVD_OSE, "\n")

The total randomized svd time is: 0.09908758299999999
The operator inference time is: 135.04317175000003
The randomized svd data error is: 0.8453425001264865
The randomized svd state error is: 0.0042674197716410555
